# Weather-based Websites Engagement Forecasting
## Objectives
The purpose of this notebook is to provide a proof that this system is capable to load real-time data (PubSub messages)

# Adding PubSub

In [26]:
import os
import ast
import pandas as pd
from google.cloud import pubsub_v1
from concurrent.futures import TimeoutError

In [27]:
credentials_path = 'weather-based-forecasting-v2-c4bde37656a7.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
timeout = 5.0

In [103]:
vec = []
def callback(message):

    print(f'Received message: {message}')
    """
    print(f'data: {message.data}')

    if message.attributes:
        print("Attributes:")
        for key in message.attributes:
            value = message.attributes.get(key)
            print(f"{key}: {value}")
    """
    message.ack()
    vec.append(message.data)

## Gogle Trends

In [42]:
subscriber = pubsub_v1.SubscriberClient()
subscription_path_GGT = 'projects/weather-based-forecasting-v2/subscriptions/nifi_trends_sub'

In [43]:
streaming_pull_future_GGT = subscriber.subscribe(subscription_path_GGT, callback=callback)
print(f'Listening for messages on {subscription_path_GGT}')

Listening for messages on projects/weather-based-forecasting-v2/subscriptions/nifi_trends_sub


## Weather Data 

In [125]:
subscriber = pubsub_v1.SubscriberClient() #Do if you didnt done it in Google trends part
subscription_path_WD = 'projects/weather-based-forecasting-v2/subscriptions/python_topic_sub'

In [126]:
streaming_pull_future_WD = subscriber.subscribe(subscription_path_WD, callback=callback)
print(f'Listening for messages on {subscription_path_WD}')

Listening for messages on projects/weather-based-forecasting-v2/subscriptions/python_topic_sub


In [ ]:
# Case 1.: Reading messages from topic using one subscriber

In [127]:
vec = []
with subscriber:
    try:
        streaming_pull_future_WD.result(timeout=10)
    except TimeoutError:

        streaming_pull_future_WD.cancel()
        streaming_pull_future_WD.result()

Received message: Message {
  data: b'{"Address": "Hamburg", "Date time": "2022-12-18 11...'
  ordering_key: '1671364020'
  attributes: {}
}
data: b'{"Address": "Hamburg", "Date time": "2022-12-18 11:47:00", "Latitude": "53.5537", "Longitude": "9.99183", "Resolved Address": "Hamburg", "Name": "Hamburg", "Wind Direction": "138.24", "Temperature": "-4.1033333333333335", "Wind Speed": "7.665", "Cloud Cover": "48.88", "Heat Index": "nan", "Chance Precipitation (%)": "0.0", "Precipitation": "0.0", "Sea Level Pressure": "1031.0", "Snow Depth": "0.0", "Snow": "0.0", "Relative Humidity": "72.30166666666666", "Wind Gust": "17.535", "Wind Chill": "-7.49", "Conditions": "Partially cloudy"}'
Received message: Message {
  data: b'{"Address": "Hamburg", "Date time": "2022-12-18 11...'
  ordering_key: '1671363900'
  attributes: {}
}
data: b'{"Address": "Hamburg", "Date time": "2022-12-18 11:45:00", "Latitude": "53.5537", "Longitude": "9.99183", "Resolved Address": "Hamburg", "Name": "Hamburg", "Wind 

In [128]:
for v in vec:
    v.decode('UTF-8')

[b'{"Address": "Hamburg", "Date time": "2022-12-18 11:47:00", "Latitude": "53.5537", "Longitude": "9.99183", "Resolved Address": "Hamburg", "Name": "Hamburg", "Wind Direction": "138.24", "Temperature": "-4.1033333333333335", "Wind Speed": "7.665", "Cloud Cover": "48.88", "Heat Index": "nan", "Chance Precipitation (%)": "0.0", "Precipitation": "0.0", "Sea Level Pressure": "1031.0", "Snow Depth": "0.0", "Snow": "0.0", "Relative Humidity": "72.30166666666666", "Wind Gust": "17.535", "Wind Chill": "-7.49", "Conditions": "Partially cloudy"}',
 b'{"Address": "Hamburg", "Date time": "2022-12-18 11:45:00", "Latitude": "53.5537", "Longitude": "9.99183", "Resolved Address": "Hamburg", "Name": "Hamburg", "Wind Direction": "138.2", "Temperature": "-4.15", "Wind Speed": "7.675", "Cloud Cover": "46.800000000000004", "Heat Index": "nan", "Chance Precipitation (%)": "0.0", "Precipitation": "0.0", "Sea Level Pressure": "1031.0", "Snow Depth": "0.0", "Snow": "0.0", "Relative Humidity": "72.425", "Wind G

In [123]:
res = ast.literal_eval(vec[0].decode('UTF-8'))
pd.DataFrame.from_dict([res])

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
0,Hamburg,2022-12-18 11:45:00,53.5537,9.99183,Hamburg,Hamburg,138.2,-4.15,7.675,46.800000000000004,nan,0.0,0.0,1031.0,0.0,0.0,72.425,17.525000000000002,-7.55,Partially cloudy


In [9]:
# Case 2.: Reading messages from topics using multiple subscribers

In [10]:
with subscriber:                                                # wrap subscriber in a 'with' block to automatically call close() when done
    try:
        streaming_pull_future_GGT.result()                          # going without a timeout will wait & block indefinitely
        streaming_pull_future_WD.result()
    except TimeoutError:
        streaming_pull_future_GGT.cancel()                          # trigger the shutdown
        streaming_pull_future_GGT.result()                          # block until the shutdown is complete
        
        streaming_pull_future_WD.cancel()                          # trigger the shutdown
        streaming_pull_future_WD.result()